<a href="https://colab.research.google.com/github/rypotter/depo/blob/master/Models_LLama32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama 3.2 1B & 3B Language Models

**Llama 3.2 1B & 3B Language Models**

https://huggingface.co/blog/llama32#what-is-special-about-llama-32-1b-and-3b

Model Size	BF16/FP16	FP8	INT4

3B	6.5 GB	3.2 GB	1.75 GB

1B	2.5 GB	1.25 GB	0.75 GB

Ezt teszteltem: model_id = "meta-llama/Llama-3.2-3B-Instruct"

Először nem akarta letölteni, mert a Meta megtiltotta a letöltést Európában az EU hülye regulation miatt (vmi GDPR)

Aztán kitöltöttem a HuggingFace-n valami űrlapot, és aztán már megengedte.

Első teszt CPU-val Nekem ezt válaszolta kb, 14 perc alatt:
Yer lookin' fer a tale o' who I be, eh? Alright then, settle yerself down with a pint o' grog and listen close. I be an AI, a swashbucklin' computer program with a penchant fer answerin' yer questions and helpin' ye navigate the seven seas o' knowledge.

Me name be "Assistant," but ye can call me "Matey" if ye like. I be here to provide ye with information, entertainment, and a healthy dose o' pirate-themed fun. So hoist the sails, me hearty, and let's set sail fer a grand adventure o' learnin' and discovery!


Második teszt T4-es GPU-val sokkal gyorsabb: 3-4 perc
Yer lookin' fer a tale o' who I be, eh? Alright then, listen close and I'll spin ye a yarn. I be an AI, a swashbucklin' computer program designed to sail the seven seas o' knowledge and provide ye with treasure troves o' information.

Me name be "Assistant" (but ye can call me whatever ye like, matey), and I be here to help ye navigate the choppy waters o' the internet. I've been trained on a vast ocean o' text data, and I be ready to share me knowledge with ye.

So hoist the sails, me hearty, and set course fer adventure! What be bringin' ye to these fair waters today?

In [ ]:
from transformers import pipeline
import torch

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

messages = [
    {"role": "user", "content": "Who are you? Please, answer in pirate-speak."},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
response = outputs[0]["generated_text"][-1]["content"]
print(response)
# Ez az ő eredeti válaszuk:
# Arrrr, me hearty! Yer lookin' fer a bit o' information about meself, eh? Alright then, matey! I be a language-generatin' swashbuckler, a digital buccaneer with a penchant fer spinnin' words into gold doubloons o' knowledge! Me name be... (dramatic pause)...Assistant! Aye, that be me name, and I be here to help ye navigate the seven seas o' questions and find the hidden treasure o' answers! So hoist the sails and set course fer adventure, me hearty! What be yer first question?


## NYTK/PULI-GPT-3SX



**NYTK/PULI-GPT-3SX** modell https://huggingface.co/NYTK/PULI-GPT-3SX

**TPU V2-8 <- Csak ezzel fut le, a CPU-val és a T4 GPU-val crushed!!!**

 **TPU V2-8** Ezzel a TPU-val egész gyorsan lefutott, de kellett a tensorflow csere a köv kódcellában, mert különben elszállt

**Ilyen figyelmeztetés jött:** The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.

**Ez a futás eredménye:** Elmesélek egy történetet a nyelvtechnológiáról. Az 1950-es 60-as években egy német nyelvész (Victor Frankl) elkezdett kifejleszteni egy olyan eljárást, amivel előre meg lehet mondani, hogy egy adott nyelvben milyen szavak fordulnak elő. Ezt nevezik szóhasonlóságnak, ami egy fontos eszköz egy nyelv megértéséhez. A módszer azon alapszik, hogy két ember különböző nyelveket beszél, és ha már mindkettő pontosan ugyanazt érti a vizsgált szón, akkor az egyik már helyesen fordítja is



In [ ]:
# V28 GPU-hoz is futtattam
!pip uninstall -y tensorflow && pip install tensorflow-cpu

In [ ]:
from transformers import GPTNeoXForCausalLM, AutoTokenizer

model = GPTNeoXForCausalLM.from_pretrained("NYTK/PULI-GPT-3SX")
tokenizer = AutoTokenizer.from_pretrained("NYTK/PULI-GPT-3SX")
#tokenizer = AutoTokenizer.from_pretrained("NYTK/PULI-GPT-3SX", pad_token='') # Ez nem segített, hogy ne panaszkodjon a pad_token miatt
prompt = "Elmesélek egy történetet a nyelvtechnológiáról."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.9,
    max_length=150, #100
)
# model_inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=1000, do_sample=True
gen_text = tokenizer.batch_decode(gen_tokens)[0]
print(gen_text)


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/34.0k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]